In [27]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

··········


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive -o nonempty

In [0]:
import pickle
import numpy as np
import tensorflow as tf
import os

In [0]:
x_train= None

for i in range(5):
    xs=[]
    ys=[]
    f = open('drive/7390final/cifar_10/data_batch_' + str(i + 1), 'rb')
    datadict = pickle.load(f, encoding='latin1')
    f.close()

    _X = datadict["data"]
    _Y = datadict['labels']
            
    _X = _X.reshape(10000,3,32,32).transpose(0,2,3,1).astype("float32")
    _Y = np.array(_Y)

    xs.append(_X)
    ys.append(_Y)
    train_data = np.concatenate(xs)
    train_labels = np.concatenate(ys)


In [0]:
train_labels

In [0]:
#Test Data
f = open('drive/7390final/cifar_10/test_batch', 'rb')
datadict = pickle.load(f, encoding='latin1')
f.close()

x_test = datadict["data"]
y_test = np.array(datadict['labels'])

x_test = np.array(x_test, dtype='float32')
test_data = x_test
test_labels = y_test

In [0]:
test_labels

In [0]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""

    input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])
    
    # Computes 32 features using a 5x5 filter 
    # with ReLU activation.
    # Input Tensor Shape: [5, 32, 32, 3]
    # Output Tensor Shape: [5, 32, 32, 32]
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    
    # First max pooling layer with a 2x2 filter 
    # and stride of 2
    # Input Tensor Shape: [5, 32, 32, 96]
    # Output Tensor Shape: [5, 16, 16, 96]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, 
                     pool_size=[2, 2], strides=2)

  
    # Computes 64 features using a 5x5 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [5, 16, 16, 32]
    # Output Tensor Shape: [5, 16, 16, 64]
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #2
    # Second max pooling layer with a 2x2 filter 
    # and stride of 2
    # Input Tensor Shape: [5, 16, 16, 64]
    # Output Tensor Shape: [5, 8, 8, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, 
                     pool_size=[2, 2], strides=2)

    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [5, 8, 8, 64]
    # Output Tensor Shape: [5, 8 * 8 * 64]
    pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64])

    # Dense Layer
    # Densely connected layer with 1024 neurons
    # Input Tensor Shape: [batch_size, 8 * 8 * 64]
    # Output Tensor Shape: [batch_size, 1024]
    dense = tf.layers.dense(inputs=pool2_flat, 
            units=1024, activation=tf.nn.relu)

    # 0.6 probability that element will be kept
    dropout = tf.layers.dropout(
        inputs=dense, rate=0.4,
        training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 10]
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


def main(unused_argv):

    # Create the Estimator
    mnist_classifier = tf.estimator.Estimator(
        model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

    # Set up logging for predictions
    # Log the values in the "Softmax" tensor with label "probabilities"
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=50)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x":train_data},
        y=train_labels,
        batch_size=128,
        num_epochs=None,
        shuffle=True)
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=20000,
        hooks=[logging_hook])

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x":test_data},
        y=test_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

In [0]:
if __name__ == "__main__":
    tf.app.run()